In [2]:
!pip install accelerate
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118
!pip install transformers accelerate bitsandbytes -q
!pip install -U bitsandbytes
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [42]:
from openai import OpenAI
import re
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import Runnable
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

In [4]:
client = OpenAI(base_url='https://api.gapapi.com/v1', api_key='sk-9AtuC5pfE0lAZ2DEGdEzRsEgPXugkaJxBNefdBCSSXgE3G1S')

In [15]:
#def llm(prompt: str) -> str:
 #   resp = client.chat.completions.create(
  #      model="gpt-4o",
 #       messages=[{"role": "user", "content": prompt}]
 #   )
 #   resp = resp.choices[0].message.content
 #   resp = re.sub("\u200c", "", resp)
  #  return resp

In [16]:
#llm("سلام")

In [23]:
class SimpleLLM(Runnable):
    def invoke(self, prompt, *args, **kwargs) -> str:
        # اگر prompt دیکشنری باشد (از PromptTemplate)
        if not isinstance(prompt, str):
            if isinstance(prompt, dict):
                # فرض: فقط یک کلید وجود دارد (متن نهایی)
                prompt = next(iter(prompt.values()))
            else:
                prompt = str(prompt)

        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )
        text = resp.choices[0].message.content
        return re.sub("\u200c", "", text)

In [24]:
llm = SimpleLLM()

In [25]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
prompt_template_name.format(cuisine = "Italian")

'I want to oprn a restaurant for Italian food. Suggest a fency name for this.'

In [26]:
chain = LLMChain(llm = llm, prompt = prompt_template_name)
chain.run("American")

'Certainly! Here’s a fancy name for your American food restaurant:\n\n**"Liberty & Fork"**\n\nThis name combines elements of elegance with a nod to American heritage ("Liberty") and the dining experience ("Fork"). It feels upscale and memorable, perfect for your concept! Let me know if you\'d like additional options.'

In [39]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
    "Return ONLY a single fancy restaurant name, no quotes, no extra words."
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """Suggest some menu items for {restaurant_name}. Return it as a separated list"""
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items)

In [40]:
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])
response = chain.run("Italian")
print(response)

Certainly! Here's a separated list of suggested menu items for "La Cucina Elegante," an Italian-inspired restaurant that focuses on elegant and refined cuisine:

**Antipasti (Starters):**  
- Burrata with Heirloom Tomatoes and Basil Oil  
- Prosciutto-Wrapped Asparagus with Balsamic Reduction  
- Stuffed Zucchini Blossoms with Ricotta and Herbs  
- Carpaccio di Manzo with Arugula and Parmesan Shavings  
- Wild Mushroom and Truffle Crostini  

**Primi (First Courses):**  
- Lobster and Saffron Risotto  
- Spinach and Ricotta Ravioli in Sage Butter Sauce  
- Pappardelle with Wild Boar Ragu  
- Squid Ink Fettuccine with Shrimp and Lemon Zest  
- Creamy Porcini Mushroom Soup  

**Secondi (Main Courses):**  
- Seared Filet Mignon with Barolo Wine Reduction  
- Herb-Crusted Rack of Lamb with Rosemary Jus  
- Grilled Branzino with Citrus-Herb Gremolata  
- Chicken Marsala with Wild Mushrooms  
- Eggplant Parmigiana with Fresh Mozzarella  

**Contorni (Side Dishes):**  
- Truffle Mashed Potato

In [46]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
    "Return ONLY a single fancy restaurant name, no quotes, no extra words."
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key = "restaurant_name")

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}."
)

food_items_chain = LLMChain(llm = llm, prompt = prompt_template_items, output_key = "menu_items")

In [47]:
chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

chain({'cuisine': 'Arabic'})

{'cuisine': 'Arabic',
 'restaurant_name': 'Saffron Mosaic',
 'menu_items': "Certainly! Here are some menu suggestions for **Saffron Mosaic**, assuming you're looking for a diverse and flavorful array of dishes that reflect sophistication and creativity. The name evokes a vibrant, international flair, so I'll include dishes inspired by global cuisines, with saffron as a potential theme ingredient:\n\n---\n\n### **Starter/Appetizers**\n1. **Saffron & Garlic Prawns**  \n   Jumbo prawns marinated with saffron, garlic, and citrus, served on a bed of microgreens.\n\n2. **Crispy Saffron Arancini**  \n   Golden risotto balls infused with saffron, stuffed with mozzarella and peas, served with a roasted tomato aioli.\n\n3. **Spiced Chickpea & Saffron Soup Shooters**  \n   Velvety chickpea soup with a hint of saffron and za'atar, served in small shooter glasses for a fun presentation.\n\n4. **Mediterranean Mezze Platter**  \n   A mosaic of hummus, baba ghanoush, marinated olives, stuffed grape le